# Разведочный анализ данных

Набор данных состоит из 1655 цифровых рентгеновских изображений коленного сустава. Исходные изображения представляют собой 8-битные изображения в оттенках серого. Каждое рентгенологическое рентгеновское изображение коленного сустава вручную классифицировано в соответствии со специальными медицинскими оценками двумя экспертами на 5 классов.

Эксперт I:

1. Normal (515 шт.)
1. Doubtful (478 шт.)
2. Mild (233 шт.)
1. Moderate (222 шт.)
1. Severe (207 шт.)


Эксперт II:

1. Normal (504 шт.)
1. Doubtful (489 шт.)
1. Mild (233 шт.)
1. Moderate (222 шт.)
1. Severe (207 шт.)

Ссылка на данные: https://tnn-hse-medtech.storage.yandexcloud.net/datasets/

## Нужные импорты

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Собираем метаданные датасета

In [39]:
import re
import hashlib
from pathlib import Path
from dataclasses import dataclass

file_pattern = re.compile(r'\w+\s+\((\d+)\)\.png')
directory = Path('~/Downloads/hse/archive (1)').expanduser()

@dataclass()
class Metadata:
    expert: str
    file_id: str
    severity: int
    relative_path: str


def parse_path(file: Path) -> Metadata:
    data_bytes = file.read_bytes()
    hash1 = hashlib.sha256(data_bytes).hexdigest()
    hash2 = hashlib.md5(data_bytes).hexdigest()
    path = file.relative_to(directory)
    severity = str(path.parent.name)
    expert = str(path.parent.parent.name)
    return Metadata(expert, f'{hash1}:{hash2}', severity, str(path))

raw_data = []
for file in directory.rglob('*.png'):
    raw_data.append(parse_path(file))
len(raw_data)

3300

# Формирует датасет


In [40]:
data = pd.DataFrame(raw_data)
data

,expert,file_id,severity,relative_path
0,MedicalExpert-II,4aaa42fc773d639e09042d8ec920bf54035f75e9c582ee...,1Doubtful,MedicalExpert-II/1Doubtful/DoubtfulG1 (203).png
1,MedicalExpert-II,b411ded0928bc1ff918a1ec0eb1c288198704cfb3eb1b4...,1Doubtful,MedicalExpert-II/1Doubtful/DoubtfulG1 (346).png
2,MedicalExpert-II,d0efac1151dccaa60cb315154a0377b5ed5dac94369781...,1Doubtful,MedicalExpert-II/1Doubtful/DoubtfulG1 (17).png
3,MedicalExpert-II,7a347ac6be61298ffad51d01fa754ca81984e7b96d7c3c...,1Doubtful,MedicalExpert-II/1Doubtful/DoubtfulG1 (254).png
4,MedicalExpert-II,290079785724f5ca891e83213f480bc449b64e33c8db21...,1Doubtful,MedicalExpert-II/1Doubtful/DoubtfulG1 (311).png
...,...,...,...,...
3295,MedicalExpert-I,7a200b679888319db15ddd968dc576334377fda09acaf9...,3Moderate,MedicalExpert-I/3Moderate/ModerateG3 (58).png
3296,MedicalExpert-I,766f164b7594b58771ac969eb3d071951ad038dd2b6c1f...,3Moderate,MedicalExpert-I/3Moderate/ModerateG3 (123).png
3297,MedicalExpert-I,f976bbf37dce6ce8baaf4b64c9809dc79363d8614c29bc...,3Moderate,MedicalExpert-I/3Moderate/ModerateG3 (135).png
3298,MedicalExpert-I,03f9f83f5335a23d629f02343797568c44fdb41c4b3ec0...,3Moderate,MedicalExpert-I/3Moderate/ModerateG3 (19).png


In [17]:
data.dtypes


expert      object
file_id     object
severity    object
dtype: object

# Нормализуем данные


In [41]:
severity_map = {
    '0Normal': 0,
    '1Doubtful': 1,
    '2Mild': 2,
    '3Moderate': 3,
    '4Severe': 4
}
data['severity'] = data['severity'].map(severity_map)


In [42]:
data.describe(include='object')

,expert,file_id,relative_path
count,3300,3300,3300
unique,2,1633,3300
top,MedicalExpert-II,197ee4a082d0ecd02972532453a8a7aaff300c2348352c...,MedicalExpert-II/1Doubtful/DoubtfulG1 (203).png
freq,1650,6,1


In [44]:
data[data['file_id'] == '197ee4a082d0ecd02972532453a8a7aaff300c2348352c866a1dba15cbb2554d:f0dc21fa5616728d830149fd3d0513bd']['relative_path']

1471    MedicalExpert-II/3Moderate/ModerateG3 (208).png
1489    MedicalExpert-II/3Moderate/ModerateG3 (203).png
1631    MedicalExpert-II/3Moderate/ModerateG3 (206).png
3121     MedicalExpert-I/3Moderate/ModerateG3 (208).png
3139     MedicalExpert-I/3Moderate/ModerateG3 (203).png
3281     MedicalExpert-I/3Moderate/ModerateG3 (206).png
Name: relative_path, dtype: object

In [34]:
data.pivot_table(index='file_id', columns='expert', values='severity')


expert,MedicalExpert-I,MedicalExpert-II
file_id,,
000555ee2250db28d7ccb076cd8ed02dc46ad1492552318a10cb98b75cf3d67c,0.0,0.0
001a4766d9a9cc32c1d7bc65b6a6d6fa6f18a5540f576e2f73217ece4b7ef50e,3.0,3.0
00439f2700963bfc89ab72a4fe7711299228d8db3fd4cb26eb92d6d8c2d1194b,2.0,2.0
005c7b80497f04a63b685eafe6797d672754684a9317ac51afbdcfab5ee481bf,2.0,2.0
00668822b6f82a52fd186a0d8583f3e276bab809b32b7153d4d0fc57c77ec7f8,1.0,1.0
...,...,...
fe910f9f5eea10fe03f6b8dbc10c4a318b55053aaa580bd4cadb44dd463be5b6,3.0,3.0
fecd647553be61596154b3766dbe39a5d3b2b9fb2edede5f0ff9c58607d69b2b,3.0,3.0
ff49383c140e3cd433c0ed3d52c923378a7958482dd4e2075401b7f4b69b8f60,1.0,1.0
